# LSTM Models with 3 different languages

### Create the configuration for the experiment

All the languages with use the same model configuration for comparing performances across them

In [1]:
%pip install conllu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras

keras.utils.set_random_seed(50)

/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next r

In [3]:
from data.preprocessor import DataPreprocessor, DataPreprocessorConfig
from trainer.trainer import TrainerConfig, Trainer
from models.base_model import ModelConfig

preprocessor_config = DataPreprocessorConfig(
    padding_type="post",
    truncation_type="post",
    remove_long_sentences=True,
    max_sequence_length=100,
)

training_config = TrainerConfig(
    epochs=20,
    batch_size=64,
    early_stopping_patience=3,
    learning_rate=1e-3,
    model_dir="saved_models",
    save_best_only=True,
)

model_config = ModelConfig(
    embedding_dim=80,
    lstm_units=128,
    bidirectional=True,
    dropout_rate=0.3,
    training_config=training_config,
)

## English

### Preprocesses the data

In [4]:
from utils import load_data

train_data, dev_data, test_data = load_data("english")

In [5]:
preprocessor = DataPreprocessor(preprocessor_config)

In [6]:
X_train, y_train = preprocessor.process_data_to_pad_sequences(
    train_data, is_train_dataset=True
)
X_dev, y_dev = preprocessor.process_data_to_pad_sequences(
    dev_data, is_train_dataset=False
)
X_test, y_test = preprocessor.process_data_to_pad_sequences(
    test_data, is_train_dataset=False
)

Sample sentence: Al - Zaman : American forces killed Shaikh Abdullah al - Ani , the preacher at the mosque in the town of Qaim , near the Syrian border .
Sample tags: PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT
Sample vectorized sentence shape: (100,)
Sample vectorized sentence: [  264    16  6152    45   293   709  1150  4860 18886   592    16 18792
     4     3  7335    35     3  7509    10     3   486     8  8930     4
   751     3  2357  1642     2     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
Sample

### Initilize the model with the config

In [7]:
from models.lstm_model import LSTMModel

english_lstm = LSTMModel(
    model_config,
    preprocessor.vocab_size,
    preprocessor.num_tags,
    preprocessor_config.max_sequence_length,
)
english_lstm.build_model()
english_lstm.compile_model()

print("Model summary:\n")
print(english_lstm.get_model().summary())

Model summary:



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 80)        │     1,574,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       214,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,792,979 (6.84 MB)

 Trainable params: 1,792,979 (6.84 MB)

 Non-trainable params: 0 (0.00 B)

None


### Training the model

In [8]:
# Initialize trainer
trainer = Trainer(training_config, english_lstm, preprocessor)

In [ ]:
# Train the model
print("Training model...\n")
trainer.train((X_train, y_train), (X_dev, y_dev))
print("Training completed.\n")

Training model...

Epoch 1/20
101/196 ━━━━━━━━━━━━━━━━━━━━ 13s 139ms/step - _masked_accuracy: 0.2766 - loss: 2.4700

### Testing the model

In [ ]:
from evaluator.evaluator import Evaluator

evaluator = Evaluator(english_lstm, preprocessor)

In [ ]:
test_metrics = evaluator.evaluate(X_test, y_test, "Test")

### Inference on new sentences

In [ ]:
from inference.predictor import Predictor

# Create predictor for inference
predictor = Predictor(english_lstm.get_model(), preprocessor)

In [ ]:
example_sentence = "Today it is cloudy"
predicted_tags = predictor.predict_sentence(example_sentence)
print(f"\nExample prediction:")
print(f"Sentence: {example_sentence}")
print(f"Predicted tags: {' '.join(predicted_tags)}")

## Spanish

In [ ]:
from utils import load_data

train_data_es, dev_data_es, test_data_es = load_data("spanish")

In [ ]:
preprocessor_es = DataPreprocessor(preprocessor_config)

In [ ]:
X_train_es, y_train_es = preprocessor_es.process_data_to_pad_sequences(
    train_data_es, is_train_dataset=True
)
X_dev_es, y_dev_es = preprocessor_es.process_data_to_pad_sequences(
    dev_data_es, is_train_dataset=False
)
X_test_es, y_test_es = preprocessor_es.process_data_to_pad_sequences(
    test_data_es, is_train_dataset=False
)

In [ ]:
from models.lstm_model import LSTMModel

es_lstm = LSTMModel(
    model_config,
    preprocessor_es.vocab_size,
    preprocessor_es.num_tags,
    preprocessor_config.max_sequence_length,
)
es_lstm.build_model()
es_lstm.compile_model()

print("Model summary:\n")
print(es_lstm.get_model().summary())

In [ ]:
trainer = Trainer(training_config, es_lstm, preprocessor_es)

In [ ]:
# Train the model
print("Training model...\n")
trainer.train((X_train_es, y_train_es), (X_dev_es, y_dev_es))
print("Training completed.\n")

In [ ]:
from evaluator.evaluator import Evaluator

evaluator_es = Evaluator(es_lstm, preprocessor_es)

In [ ]:
test_metrics_es = evaluator_es.evaluate(X_test_es, y_test_es, "Test")

### Inference on new sentences

In [ ]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_es = Predictor(es_lstm.get_model(), preprocessor_es)

In [ ]:
example_sentence = "Hoy el tiempo es soleado ."
predicted_tags = predictor_es.predict_sentence(example_sentence)
print(f"\nExample prediction:")
print(f"Sentence: {example_sentence}")
print(f"Predicted tags: {' '.join(predicted_tags)}")

## German

In [ ]:
from utils import load_data

train_data_ge, dev_data_ge, test_data_ge = load_data("german")

In [ ]:
preprocessor_ge = DataPreprocessor(preprocessor_config)

In [ ]:
X_train_ge, y_train_ge = preprocessor_ge.process_data_to_pad_sequences(
    train_data_ge, is_train_dataset=True
)
X_dev_ge, y_dev_ge = preprocessor_ge.process_data_to_pad_sequences(
    dev_data_ge, is_train_dataset=False
)
X_test_ge, y_test_ge = preprocessor_ge.process_data_to_pad_sequences(
    test_data_ge, is_train_dataset=False
)

In [ ]:
from models.lstm_model import LSTMModel

ge_lstm = LSTMModel(
    model_config,
    preprocessor_ge.vocab_size,
    preprocessor_ge.num_tags,
    preprocessor_config.max_sequence_length,
)
ge_lstm.build_model()
ge_lstm.compile_model()

print("Model summary:\n")
print(ge_lstm.get_model().summary())

In [ ]:
trainer_ge = Trainer(training_config, ge_lstm, preprocessor_ge)

In [ ]:
# Train the model
print("Training model...\n")
trainer_ge.train((X_train_ge, y_train_ge), (X_dev_ge, y_dev_ge))
print("Training completed.\n")

In [ ]:
from evaluator.evaluator import Evaluator

evaluator_ge = Evaluator(ge_lstm, preprocessor_ge)

In [ ]:
test_metrics_ge = evaluator_ge.evaluate(X_test_ge, y_test_ge, "Test")

### Inference on new sentences

In [ ]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_ge = Predictor(ge_lstm.get_model(), preprocessor_ge)

In [ ]:
example_sentence = "Heute ist es wolkig ."
predicted_tags = predictor_es.predict_sentence(example_sentence)
print(f"\nExample prediction:")
print(f"Sentence: {example_sentence}")
print(f"Predicted tags: {' '.join(predicted_tags)}")